# Set paths to training data OK and NOK

In [1]:
from IPython import get_ipython
# Initialize
DATASET_OK_PATH = './data/ErgosFabric/OK'
DATASET_NOK_PATH = './data/ErgosFabric/NOK'
DATASET_MASK_PATH = './data/ErgosFabric/Mask'

ZIP_OK_OUTDIR_PATH = './data/ErgosFabric_OK.zip'
ZIP_NOK_OUTDIR_PATH = './data/ErgosFabric_NOK.zip'
OUTDIR_PATH = './runs/ErgosFabric'

# Width and height to scale the training image
IMAGE_SIZE = 256


# Training parameters

In [2]:
# function for invoking other command line functions
def call_function(function, **kwargs):
    """
    ChatGPT helped, looks like bs
    """
    command = f"%run {function} {' '.join([f'--{k} {v}' for k, v in kwargs.items()])}"
    print(f"Invoked command:\n {command}")
    
    ipython = get_ipython()
    ipython.magic(f"%run {function} {' '.join([f'--{k} {v}' for k, v in kwargs.items()])}")

# Generate .zip from dataset for training
Create two .zip files.
 - First set of OK images to train generation of non-defect samples
 - Second set of NOK images and masks specifying the defect to train generation of defect samples

In [ ]:
kwargs_OK_DATA = {"source": DATASET_OK_PATH, "dest":ZIP_OK_OUTDIR_PATH, "width": IMAGE_SIZE, "height": IMAGE_SIZE}
kwargs_NOK_DATA = {"source": DATASET_NOK_PATH, "source-mask": DATASET_MASK_PATH, "dest": ZIP_NOK_OUTDIR_PATH, 'width': IMAGE_SIZE, "height": IMAGE_SIZE}

In [ ]:
# Create OK training .zip dataset
call_function('dataset_tool.py', kwargs_OK_DATA)

In [ ]:
# Create NOK training .zip dataset
call_function('dataset_tool.py', kwargs_NOK_DATA)

In [3]:
NUM_GPU = 1
# How many thousands of images
KIMG = 750
SNAP = 50
# Path to network to resume training, None otherwise
RESUME = None
# Image transfer method...
TRANSFER = None

In [4]:
# Arguments for OK training 
kwargs_OK = {'data': ZIP_OK_OUTDIR_PATH, 'outdir': OUTDIR_PATH, 'gpus': NUM_GPU, 'kimg': KIMG, 'snap': SNAP}
# Arguments for NOK training 
kwargs_NG = {'data': ZIP_NOK_OUTDIR_PATH, 'outdir': OUTDIR_PATH, 'gpus': NUM_GPU, 'kimg': KIMG, 'snap': SNAP, 'resume': RESUME, 'transfer': TRANSFER}

In [5]:
call_function('train.py', **kwargs_OK)

c:\Users\pekat\AppData\Local\pypoetry\Cache\virtualenvs\bachelors-python37-i6IQsbMd-py3.7\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./data/ErgosFabric_OK.zip",
    "use_labels": false,
    "max_size": 61,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256,
      "no_round": false,
      "tanh_mask": "late",
      "tanh_k": 10.0
    },
    "transfer": "none",
    "mask_threshold": 0.0
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mappi